# Finding the similarity between title and product text using word2vec and cosine similarity

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
from gensim.models import Word2Vec
import nltk
# nltk.download('wordnet')
stemmer = SnowballStemmer('english')
import unicodedata
from numpy import dot
from numpy.linalg import norm
# Lemmatize with POS Tag
from nltk.corpus import wordnet
import gc
import cudf
import cupy
import torch
%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

In [ ]:
train_df = pd.read_csv('/kaggle/input/shopee-product-matching/train.csv')
DATA_PATH = '/kaggle/input/shopee-product-matching/'

In [ ]:
test_df = pd.read_csv('../input/shopee-product-matching/test.csv')
df_cu = cudf.DataFrame(test_df)

# Data preprocessing

In [ ]:
def cleanData(dataParse):
    data = unicodedata.normalize('NFKC', dataParse)
    data = re.sub(r'【.*】', '', data)
    data = re.sub(r'\[.*\]', '', data)
    data = re.sub(r'「.*」', '', data)
    data = re.sub(r'\(.*\)', '', data)
    data = re.sub(r'\<.*\>', '', data)
    data = re.sub(r'[※@◎].*$', '', data)
    return data.lower() #Returns the parsed tweets


def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(cleanData(text), pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            if token == 'xxxx':
                continue
            result.append(lemmatize_stemming(token))
    
    return result

# Load model parameter for traing

In [ ]:
# Load word2vec model

w2v_model = Word2Vec.load("../input/word2vec-model/word2vec_model")

# Find the similarity between two vector

In [ ]:
# Generate the average word2vec for the each title description
non_dup_train_df=test_df.drop_duplicates(subset=['title'])
non_dup_train_df=non_dup_train_df.reset_index()
def vectors_test(): #test_df
    
    # Creating a list for storing the vectors (description into vectors)
    word_embeddings_test = []
    test_processed_docs = non_dup_train_df['title'].map(preprocess)
    
    # Reading the each book description 
    for line in test_processed_docs:
        avgword2vec = None
        count = 0
        for word in line:
            if word in w2v_model.wv:
                count += 1
                if avgword2vec is None:
                    avgword2vec = w2v_model.wv[word]
                else:
                    avgword2vec = avgword2vec + w2v_model.wv[word]
                
        if avgword2vec is not None:
            avgword2vec = avgword2vec / count
            word_embeddings_test.append(avgword2vec)
        else:
            word_embeddings_test.append(np.array([0]*50, dtype='float32'))
    return word_embeddings_test

In [ ]:
word_embeddings_test = cupy.array(vectors_test(), dtype=cupy.float32)#test_df

In [ ]:
def similarity1():
    preds = []
    CHUNK = 1024*4

    print('Finding similar titles...')
    CTS = len(test_df)//CHUNK
    if len(test_df)%CHUNK!=0: CTS += 1
    for j in range( CTS ):

        a = j*CHUNK
        b = (j+1)*CHUNK
        b = min(b,len(test_df))
        print('chunk',a,'to',b)

        # COSINE SIMILARITY DISTANCE
        cts = cupy.matmul( word_embeddings_test, word_embeddings_test[a:b].T).T        

        for k in range(b-a):
            IDX = cupy.where(cts[k,]>9)[0]
            o = test_df.iloc[cupy.asnumpy(IDX)].posting_id.values
            preds.append(o)

    
    return preds

In [ ]:
def combine_predictions(row):
    return ' '.join( np.unique(row))

In [ ]:
non_dup_train_df['test_matches']=similarity1()
non_dup_train_df['matches'] = non_dup_train_df['test_matches'].apply(combine_predictions)
del word_embeddings_test
del w2v_model
gc.collect()
non_dup_train_df[['posting_id', 'matches']].to_csv('submission.csv', index = False)

In [ ]:
non_dup_train_df[['posting_id', 'matches']].head()

# keep learning and enrich your intuition.Don't forgot to upvote !! :)